In [1]:
import pandas as pd
import numpy as np

import timeit
import nltk

import scipy.spatial.distance as distance

In [2]:
df_aux = pd.read_csv('dataset', sep="\n", names=['p1'])
#Cogemos la 1ra mitad del dataset y para la columna 1
df = df_aux.iloc[:147]

In [3]:
#Cogemos las 150 empezando desde atras, reseteamos el index para que empiece desde 0, y nombramos con algo entendible
df_aux = df_aux[-150:]
df_aux = df_aux.reset_index()
df_aux.columns = ['index','p2']

In [4]:
df['p2'] = df_aux['p2']

In [10]:
#Tokenizer

df2 = df.copy()
start_time = timeit.default_timer()
df2['p1'] = pd.Series([nltk.word_tokenize(x[0]) for x in df.itertuples(index=False)], index=df.index)
df2['p2'] = pd.Series([nltk.word_tokenize(x[1]) for x in df.itertuples(index=False)], index=df.index)
df2["avgQ1Q2"] = -1.0
print(timeit.default_timer() - start_time)

0.12199123100072029


In [12]:
#Word2Vec
import gensim
start_time = timeit.default_timer()

model = gensim.models.KeyedVectors.load('../../quora/data/GoogleNews-vectors-negative300.bin.gz', mmap=None)

print(timeit.default_timer() - start_time)

Using TensorFlow backend.


32.349580791000335


In [16]:
words_not_founded = set()

def avg_feature_vector(words, num_features, model):
    """ Metodo que saca el vector medio de un cantidad de palabras dadas 
        word: palabras separadas de las que queremos obtener el vector medio
        num_features: El tam del vector que representa a una palabra del modelo usado
    """
    """https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt"""
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    global words_not_founded
    
    for word in words:
        if word in model:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
        else:
            words_not_founded.add(word)
    if(nwords > 0):
        featureVec = np.divide(featureVec, nwords)
    else:
        featureVec.fill(1)
    return featureVec

In [18]:
df2['avgQ1Q2'] = df2.apply(lambda row: distance.cosine(
    avg_feature_vector(df2["p1"][row[0]], num_features=300, model=model),
    avg_feature_vector(df2["p2"][row[0]], num_features=300, model=model)), axis=1)

KeyError: ("None of [['Splunk', 'Brings', '#', 'MachineLearning', 'to', '#', 'DataAnalytics', '-', 'https', ':', '//t.co/FeBWPlJBo6', '#', 'Analytics', '#', 'BigData', '#', 'ArtificialIntelligence', '#', 'DeepLearning', '*-*', 'OttLegalRebels']] are in the [index]", 'occurred at index 0')